In [5]:
from typing import List
from collections import defaultdict
from functools import lru_cache

class Solution:
    def catMouseGame(self, graph: List[List[int]]) -> int:
        @lru_cache(None)
        def dfs(mouse, cat, step):
            if step > len(graph) * 2:
                return 0
            if cat == mouse:
                return 2
            if mouse == 0:
                return 1
            
            if step % 2 == 0:# 老鼠行动
                draw = False
                for nm in graph[mouse]:
                    ans = dfs(nm, cat, step + 1)
                    if ans == 0:
                        draw = True
                    elif ans == 1:
                        return 1
                return 0 if draw else 2
            else:
                draw = False
                for nc in graph[cat]:
                    if nc == 0:
                        continue
                    ans = dfs(mouse, nc, step + 1)
                    if ans == 0:
                        draw = True
                    elif ans == 2:
                        return 2
                return 0 if draw else 1
        
        return dfs(1, 2, 0)

In [20]:
from typing import List
from collections import defaultdict, deque

class Solution:
    def catMouseGame(self, graph: List[List[int]]) -> int:
        # 用 已知 推 未知
        def getPreState(m, c, t):
            pos = []
            if t == 1: # 如果当前轮到老鼠走，那么前一步是轮到猫走
                for nc in graph[c]:
                    if nc == 0:
                        continue
                    pos.append((m, nc, 2))
            else:
                for nm in graph[m]:
                    pos.append((nm, c, 1))
            return pos
        
        def mustLoss(m, c, t):
            if t == 1:
                for nm in graph[m]:
                    if res[(nm, c, 2)] != 2:
                        return False
            else:
                for nc in graph[c]:
                    if nc == 0:
                        continue
                    if res[(m, nc, 1)] != 1:
                        return False
            return True
        
        res = defaultdict(int) # 记录每个状态的输赢
        queue = deque()
        n = len(graph)
        for t in range(1, 3): # 1: mouse move, 2: cat move
            for i in range(1, n):
                res[(0, i, t)] = 1 # 如果是1，就是老鼠赢
                queue.append((0, i, t))
                res[(i, i, t)] = 2 # 如果是2，就是猫赢
                queue.append((i, i, t))
                
        while queue:
            m, c, t = queue.popleft()
            ans = res[(m, c, t)] # 当前状态的结果，是已经知道谁赢谁输的
            
            for pre in getPreState(m, c, t): # 用当前的状态推导之前的状态
                m2, c2, t2 = pre # 先前的一个状态
                
                if res[pre] != 0:
                    continue
                
                if ans == t2:
                    res[pre] = ans
                    queue.append(pre)
                elif mustLoss(m2, c2, t2):
                    res[pre] = 3 - t2
                    queue.append(pre)
        
        return res[(1, 2, 1)] # 初始位置为(1,2,1)

In [21]:
solution = Solution()
solution.catMouseGame([[1,3],[0],[3],[0,2]])

1